In [ ]:
plot = TRUE
# don't need to set a working directory.  Just use os.curdir.
# don't need a repo

In [4]:
import pandas as pd
import numpy as np
import rasterio as rio
import os, glob

In [ ]:
# skipping these this for now.

In [ ]:
#create folders 'Temp_RModelOutputsTiffs' and 'R_FinalOutputs' if they don't exist in current working directory.
subdir1 = 'Temp_RModelOutputsTiffs'
subdir2 = 'R_FinalOutputs'

In [ ]:
print('Reading Region Boundaries...')
GoodeR_Boundaries_Region =  pd.read_csv(GoodeR_Boundaries_Region.csv) # with ID and region as int
print('Reading Training Points...')
TrainingPoints = pd.read_csv(TrainingPointsFull.csv) # with ID and class as int
print('Reading Loss Mask...')
LossMaskFull = pd.read_csv(LossMaskFull.csv)

In [45]:
GoodePy = range(1,6961897) # I see no reason for this to be a dataframe

In [ ]:
#Optional code to read data from a previous run (saved as GoodeR_SecondaryData.csv and TrainingPoints_PrimaryData.csv).  Maybe add later.

In [32]:
# PrimaryFileList = list of file names of TIFs in /R_ModelInputs_PrimaryData/    ###~~~ Originally PrimaryFileList was a df, not a list.  We use a list here.
PrimaryFileList = glob.glob('R_ModelInputs_PrimaryData/*.tif')

In [33]:
# PrimaryVariables = list of file names with subfolder, 'Goode_' & '.tif' removed from the strings in PrimaryFileList
PrimaryVariables = PrimaryFileList
for idx, file in enumerate(PrimaryVariables):
    PrimaryVariables[idx] = file.replace('R_ModelInputs_PrimaryData\\', '').replace('Goode_', '').replace('.tif', '')

In [31]:
# I'm pretty sure these are identical to PrimaryVariables and PrimaryFileList, but I'll soon find out.

['_release_R_ModelInputs_SecondaryData\\Fire_Brightness_10kMax.tif',
 '_release_R_ModelInputs_SecondaryData\\Fire_Brightness_10kMax1kMean.tif',
 '_release_R_ModelInputs_SecondaryData\\Fire_Brightness_10kMax1kSum.tif',
 '_release_R_ModelInputs_SecondaryData\\Fire_Brightness_10kMean.tif',
 '_release_R_ModelInputs_SecondaryData\\Fire_Brightness_10kMean1kMax.tif',
 '_release_R_ModelInputs_SecondaryData\\Fire_Brightness_10kMean1kSum.tif',
 '_release_R_ModelInputs_SecondaryData\\Fire_Brightness_10kSum.tif',
 '_release_R_ModelInputs_SecondaryData\\Fire_Count_10kMax.tif',
 '_release_R_ModelInputs_SecondaryData\\Fire_Count_10kMax1kMean.tif',
 '_release_R_ModelInputs_SecondaryData\\Fire_Count_10kMax1kSum.tif',
 '_release_R_ModelInputs_SecondaryData\\Fire_Count_10kMean.tif',
 '_release_R_ModelInputs_SecondaryData\\Fire_Count_10kMean1kMax.tif',
 '_release_R_ModelInputs_SecondaryData\\Fire_Count_10kMean1kSum.tif',
 '_release_R_ModelInputs_SecondaryData\\Fire_Count_10kSum.tif',
 '_release_R_ModelInp

In [ ]:
print('Checking inputs for TrainingPoints_PrimaryData...')

In [ ]:
# iterate across PrimaryFileList, ensuring correct input resolution of 1737x4008 on each tiff
for name in PrimaryFileList:
    data = rio.open(name)
    if data.width != 4008:
        print(f"{data.name} has {data.width} columns, not 4008!")
        errors = TRUE
    elif data.height != 1737:
        print(f"{data.name} has {data.height} rows, now 1737!")
        errors = TRUE
## ideally, check other metadata, like crs

In [ ]:
# Not necessary in a notebook.  Fix the problem and start over if there are errors.  Keep going if there aren't.

say('Calculating Training Points...')

In [ ]:
# Convert class values from into one-hot table.
training_one_hot = pd.get_dummies(TrainingPoints[['Training.Class']], prefix='Class')
TrainingPoints = pd.merge(TrainingPoints, training_one_hot, right_index=TRUE, left_index=TRUE)

In [ ]:
# Create TrainingPoints_PrimaryData by filter out classes 6 & 7 from TrainingPoints.  (Philip's code only filters 7 initially, but I think it ends up not mattering later).
TrainingPoints_PrimaryData = TrainingPoints.iloc[TrainingPoints['Training.Class'] < 6]

In [ ]:
for name in PrimaryFileList: #Original specifies the column 'Filename' because it creates PrimaryFileList as a dataframe
    print('Reading ', name)
    data = rio.open(name)
    name2 = data.name.replace('Goode_', '')
    df = pd.DataFrame({name2: data}, index = GoodePy)
    TrainingPoints_PrimaryData = pd.merge(TrainingPoints_PrimaryData, df, how='left', on=name2)

TrainingPoints_PrimaryData.fillna(0).drop_duplicates()

In [51]:
regions = range(1,8)
driverNames = ['Deforestation', 'Shifting.Agriculture', 'TreeFarm.ForestryOther', 'Wildfire', 'Urban']
drivers = range(1,6)
ModelOutput = []
ModelOutput.Regional = []
if plots:
   #make plots
# something with importance???

## This is where the magic happens!